# Manage metadata using ontologies

The standalone package [Bionty](https://lamin.ai/docs/bionty/) helps with curating metadata using ontologies.

If you only work with pre-defined ontologies (public or in-house), Bionty is sufficient - check out its [docs](https://lamin.ai/docs/bionty/)!

If you'd like to maintain in-house along with public ontologies, we recommend managing them using `lnschema_bionty`'s ORMs.

In this guide, you'll see how to use `bionty` and `lnschema_bionty` in conjunction.

In [ ]:
import lamindb as ln
import bionty as bt
import lnschema_bionty as lnbt

## Lookup a public ontology entry

In [ ]:
cell_type_lookup = bt.CellType().lookup()

In [ ]:
cell_type_lookup.gamma_delta_T_cell

## Create a record for an in-house ontology

In [ ]:
cell_type = lnbt.CellType(cell_type_lookup.gamma_delta_T_cell)

cell_type

(Note: The id here is a deterministic hash of the ontology entry.)

You can add it to the DB to seed an in-house ontology:

In [ ]:
ln.add(cell_type)

In [ ]:
ln.select(lnbt.CellType).df()

You can now work with a lookup object with much less terms: `lnbt.CellType.lookup()`

## Parse records from data

Often, you want to parse records from data and map it onto a reference. {func}`~lamindb.parse` takes any iterable and maps it on your in-house reference.

Consider a DataFrame-based example:

In [ ]:
adata = ln.dev.datasets.anndata_with_obs()

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.cell_type.value_counts()

Here, we're using the cell type name column:

In [ ]:
cell_types = ln.parse(adata.obs.cell_type, lnbt.CellType.name)

cell_types

Here, we're using the cell type id column (and observe that "my new cell type" is skipped):

In [ ]:
ln.parse(adata.obs.cell_type_id, lnbt.CellType.ontology_id)

If we're happy with `cell_types`, we commit them to the DB:

In [ ]:
ln.add(cell_types);

Our in-house ontology grew a bit:

In [ ]:
ln.select(lnbt.CellType).df()

Of course, this also works for any other entity in Bionty.

## Automated tracking of underlying ontology versions

Under-the-hood, ontology-versions are tracked:

In [ ]:
ln.select(lnbt.dev.BiontyVersions).join(lnbt.dev.CurrentBiontyVersions).df()